In [ ]:
"""
%pip install matplotlib==3.10.1
%pip install pennylane==0.42.3
%pip install numpy==1.26.4
%pip install pandas==2.2.2
%pip install scipy==1.15.2
%pip install basis-set-exchange
%pip install qiskit-aer==0.17.1
%pip install pennylane-qiskit==0.40.1
%pip install qiskit-aer-gpu==0.15.1
"""

'\n%pip install matplotlib==3.10.1\n%pip install pennylane==0.42.3\n%pip install numpy==1.26.4\n%pip install pandas==2.2.2\n%pip install scipy==1.15.2\n%pip install basis-set-exchange\n%pip install qiskit-aer==0.17.1\n%pip install pennylane-qiskit==0.40.1\n%pip install qiskit-aer-gpu==0.15.1\n'

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# 1º: Create the Molecule and Hamiltonian

In the code cells below, we create the molecular geometry of H2 with the radial distance of 1,401 a.u. with the optimal internuclear distance of Szabo and Ostlund. We will use the STO-3G minimal basis set by importing the parameters of the basis set exchange functions. Pennylane allows us to construct the Hamiltonian from the second quantization with the Jordan-Wigner transformation.

In [ ]:
from scipy.optimize import minimize
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt
import pennylane as qml
import pandas as pd
import numpy as np
import time
import json

symbols = ["H", "H"]
coordinates = np.array([[0.0, 0.0, 0], [0.0, 0.0, 1.401]])
molecule = qml.qchem.Molecule(symbols, coordinates, basis_name='STO-3G', load_data=True, unit='bohr')
H, qubits = qml.qchem.molecular_hamiltonian(molecule)

print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

Number of qubits =  4
The Hamiltonian is  -0.09883485860187924 * I([0, 1, 2, 3]) + 0.17120123803197834 * Z(0) + 0.17120123803197845 * Z(1) + 0.16862327620358059 * (Z(0) @ Z(1)) + -0.22279639115527738 * Z(2) + 0.12054612718324412 * (Z(0) @ Z(2)) + 0.16586801098505832 * (Z(1) @ Z(2)) + 0.045321883801814206 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.045321883801814206 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.045321883801814206 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 0.045321883801814206 * (X(0) @ Y(1) @ Y(2) @ X(3)) + -0.22279639115527738 * Z(3) + 0.16586801098505832 * (Z(0) @ Z(3)) + 0.12054612718324412 * (Z(1) @ Z(3)) + 0.17434948668373768 * (Z(2) @ Z(3))


# 2º: Calculate the Number of Shots Necessary for the Experiment

In the code cells below, we calculate the number of shots required for our noise-free experiment using the COBYLA optimizer. This calculation is based on sampling formulas and a specified precision, denoted by 𝜖. After determining the required number of shots, we round it to 4,000,000. Increasing the precision will result in longer computation times and may also require more memory.

In [ ]:
e = 1e-3
cj = H.terms()[0]

sum_squarred = sum([abs(float(c)) for c in cj])**2

shots_necessary = round(1/(e**2)*(sum_squarred))
shots_necessary

3935933

In [ ]:
shots_necessary=4000000

# 3º: Configure the Simulator

In the cells below, we configure a noise-free Qiskit Aer simulator using the PennyLane-Qiskit plugin. Our goal is to simulate the UCCSD ansatz with PennyLane. The quantum circuit is designed to compute the expected value of the Hamiltonian by collecting a large number of samples obtained from the shots. For these simulations, we will leverage the GPU to accelerate the computations.

In [ ]:
aer_simulator = AerSimulator(method='statevector', device='GPU')

In [ ]:
dev = qml.device("qiskit.aer", backend=aer_simulator, wires=qubits, shots=shots_necessary)

In [ ]:
electrons = 2
wires = range(qubits)
hf_state = qml.qchem.hf_state(electrons, qubits)
singles, doubles = qml.qchem.excitations(electrons, qubits)
s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

In [ ]:
@qml.qnode(dev)
def circuit(params):
    qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
    return qml.expval(H)

In [ ]:
energy_data = []
params_data = []

In [ ]:
def cost_fn(param):
    energie = circuit(param)
    energy_data.append(energie)
    params_data.append(param)
    return energie

# 4º: Configure the optimizer and run the circuit

In the code cells below, we configure the COBYLA optimizer, a widely used optimizer in VQE experiments that performs well in noise-free environments. It is set to a maximum of 250 iterations (typically converging in 70 to 71 steps) with a tolerance of 1×10^−8. We also provide the initial parameter angles to be optimized. The circuit is executed iteratively until convergence is achieved.

In [ ]:
t0 = time.perf_counter()
def callback(theta):
    current_energy = energy_data[-1].mean().item()
    current_params = params_data[-1]
    step = len(energy_data)
    print(f"Step {step}: Energy = {current_energy:.8f} Ha | θ = {current_params} rad")

init_params = np.zeros(len(singles) + len(doubles))

res = minimize(
    fun=lambda x: cost_fn(x).item(),
    x0=init_params,
    method='COBYLA',
    callback=callback,
    options={'maxiter': 250, 'tol': 1e-8}
)
t1 = time.perf_counter()

print("\nFinal Energy Estimate:", res.fun)
print("Optimal Parameter:", res.x)
print(f"Execution Time: {t1-t0}")

Step 1: Energy = -1.11664369 Ha | θ = [0. 0. 0.] rad
Step 2: Energy = -0.94065920 Ha | θ = [1. 0. 0.] rad
Step 3: Energy = -0.94081979 Ha | θ = [0. 1. 0.] rad
Step 4: Energy = -0.90685356 Ha | θ = [0. 0. 1.] rad
Step 5: Energy = -0.74155418 Ha | θ = [-0.54079619 -0.5403027  -0.64468013] rad
Step 6: Energy = -0.98311465 Ha | θ = [-0.27039809 -0.27015135 -0.32234007] rad
Step 7: Energy = -1.09657734 Ha | θ = [ 0.23961    -0.04581363 -0.05466406] rad
Step 8: Energy = -1.09783320 Ha | θ = [ 0.          0.09580297 -0.08029191] rad
Step 9: Energy = -1.13629531 Ha | θ = [0.00652138 0.06128262 0.24228478] rad
Step 10: Energy = -1.11868284 Ha | θ = [-0.10876621 -0.0917229   0.40290259] rad
Step 11: Energy = -1.13261907 Ha | θ = [0.12481998 0.03066311 0.2686075 ] rad
Step 12: Energy = -1.12916846 Ha | θ = [0.00699814 0.1822365  0.27382731] rad
Step 13: Energy = -1.13602888 Ha | θ = [-0.02423353  0.02679569  0.28436847] rad
Step 14: Energy = -1.13679921 Ha | θ = [-0.01549929  0.02812552  0.194098

# 5º: Save the Data for Later Analysis

In [ ]:
df = pd.DataFrame({
    "energies": energy_data,
    "params": params_data
})

df["energies_json"] = df["energies"].apply(lambda arr: json.dumps(arr.tolist()))
df["params_json"]  = df["params"].apply(lambda arr: json.dumps(arr.tolist()))

In [ ]:
df.to_csv("COBYLA_NOISE_FREE.csv", columns=["energies_json","params_json"], index=False)